In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql



In [3]:
#uploading csv
df = pd.read_csv("C:/Users/Bhuvaneshwari/Downloads/synthetic_client_queries - synthetic_client_queries.csv")
df

,query_id,client_email,client_mobile,query_heading,query_description,status,date_raised,date_closed,@
0,Q0001,joshuaoneill@example.net,6638686049,Bug Report,Tab focus jumps incorrectly.,Closed,2025-02-26,2025-02-27,NaN
1,Q0002,edward91@example.net,6369365581,Bug Report,Form validation not working properly.,Closed,2025-07-22,2025-08-01,NaN
2,Q0003,cookmason@example.org,2686615523,Account Suspension,Need help lifting restrictions.,Closed,2025-06-18,2025-06-19,NaN
3,Q0004,wtaylor@example.com,4753393384,Data Export,Need monthly data dump in CSV.,Closed,2025-02-25,2025-03-07,NaN
4,Q0005,rmoore@example.net,8523755943,UI Feedback,Icons not intuitive.,Closed,2025-06-19,2025-06-20,NaN
...,...,...,...,...,...,...,...,...,...
5195,Q5196,fhenry@example.org,2758293072,Payment Failure,EMI option not available anymore.,Closed,2025-05-03,2025-05-09,NaN
5196,Q5197,robert82@example.net,2400038786,Technical Support,Freezing during screen load.,Closed,2025-05-17,2025-05-20,NaN
5197,Q5198,mclaughlinmichael@example.net,312901852,Login Issue,Forgot my password and reset isn’t working.,Closed,2025-06-25,2025-07-04,NaN
5198,Q5199,vaughnkristen@example.org,795334441,UI Feedback,Graphs too compressed on mobile view.,Closed,2025-07-15,2025-07-24,NaN


In [4]:
df.head()

,query_id,client_email,client_mobile,query_heading,query_description,status,date_raised,date_closed,@
0,Q0001,joshuaoneill@example.net,6638686049,Bug Report,Tab focus jumps incorrectly.,Closed,2025-02-26,2025-02-27,NaN
1,Q0002,edward91@example.net,6369365581,Bug Report,Form validation not working properly.,Closed,2025-07-22,2025-08-01,NaN
2,Q0003,cookmason@example.org,2686615523,Account Suspension,Need help lifting restrictions.,Closed,2025-06-18,2025-06-19,NaN
3,Q0004,wtaylor@example.com,4753393384,Data Export,Need monthly data dump in CSV.,Closed,2025-02-25,2025-03-07,NaN
4,Q0005,rmoore@example.net,8523755943,UI Feedback,Icons not intuitive.,Closed,2025-06-19,2025-06-20,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   query_id           5200 non-null   object 
 1   client_email       5200 non-null   object 
 2   client_mobile      5200 non-null   int64  
 3   query_heading      5200 non-null   object 
 4   query_description  5200 non-null   object 
 5   status             5200 non-null   object 
 6   date_raised        5200 non-null   object 
 7   date_closed        5146 non-null   object 
 8   @                  0 non-null      float64
dtypes: float64(1), int64(1), object(7)
memory usage: 365.8+ KB


In [6]:
#view columns
df.columns

Index(['query_id', 'client_email', 'client_mobile', 'query_heading',
       'query_description', 'status', 'date_raised', 'date_closed', '@'],
      dtype='object')

In [7]:
# check how many nulls in each column
df.isnull().sum()

query_id                0
client_email            0
client_mobile           0
query_heading           0
query_description       0
status                  0
date_raised             0
date_closed            54
@                    5200
dtype: int64

In [8]:
#dropping an entire column of null values
df.drop(columns = ["@"], inplace = True)

In [9]:
#removing whitespaces
df["client_email"] = df["client_email"].str.strip().str.lower()

In [10]:
#formatting mobile num
df["client_mobile"] = df["client_mobile"].astype(str).str.replace(r"\D","9845789520",regex = True)
df = df[df["client_mobile"].str.len()==10]

In [11]:
#formatting dates
df = df.copy() 
df.loc[:, "date_closed"] = pd.to_datetime(df["date_closed"])

In [12]:
df = df.copy() 
df.loc[:, "date_raised"] = pd.to_datetime(df["date_raised"])

In [13]:
df.isnull().sum().sum()

np.int64(52)

In [14]:
#check dup
df.duplicated().sum()

np.int64(0)

In [15]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 4682 entries, 0 to 5199
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query_id           4682 non-null   object
 1   client_email       4682 non-null   object
 2   client_mobile      4682 non-null   object
 3   query_heading      4682 non-null   object
 4   query_description  4682 non-null   object
 5   status             4682 non-null   object
 6   date_raised        4682 non-null   object
 7   date_closed        4630 non-null   object
dtypes: object(8)
memory usage: 329.2+ KB


,query_id,client_email,client_mobile,query_heading,query_description,status,date_raised,date_closed
0,Q0001,joshuaoneill@example.net,6638686049,Bug Report,Tab focus jumps incorrectly.,Closed,2025-02-26 00:00:00,2025-02-27 00:00:00
1,Q0002,edward91@example.net,6369365581,Bug Report,Form validation not working properly.,Closed,2025-07-22 00:00:00,2025-08-01 00:00:00
2,Q0003,cookmason@example.org,2686615523,Account Suspension,Need help lifting restrictions.,Closed,2025-06-18 00:00:00,2025-06-19 00:00:00
3,Q0004,wtaylor@example.com,4753393384,Data Export,Need monthly data dump in CSV.,Closed,2025-02-25 00:00:00,2025-03-07 00:00:00
4,Q0005,rmoore@example.net,8523755943,UI Feedback,Icons not intuitive.,Closed,2025-06-19 00:00:00,2025-06-20 00:00:00


In [ ]:
#connecting DB
import pymysql
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="123456789",
    database="client_query"
)
cursor = conn.cursor()

In [ ]:
#execute 
data = df[["query_id", "client_email", "client_mobile", "query_heading", "query_description", "status", "date_raised", "date_closed"]].values.tolist()

cursor.executemany(
	"""
	INSERT INTO client (query_id, client_email, client_mobile, query_heading, query_description, status, date_raised, date_closed)
	VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
	""",
	data
)
conn.commit()